In [243]:
import spacy
import de_core_news_sm
nlp = de_core_news_sm.load()
from spacy.matcher import Matcher
from charsplit import Splitter
import string
splitter = Splitter()
import pandas as pd
import jellyfish
import random

In [244]:
def clear_special_characters(s1, s2):
    invalidcharacters = set(string.punctuation)
    if any(char in invalidcharacters for char in s1):
        s1_ = s1.lower().translate(str.maketrans('', '', string.punctuation))
    else:
        s1_ = s1
    if any(char in invalidcharacters for char in s2):
        s2_ = s2.lower().translate(str.maketrans('', '', string.punctuation))
    else:
        s2_ = s2
    return s1_, s2_

In [245]:
def stop_words_handling(term):
    splitted_term = term.split()
    stop_words = set(["for", "and", "of", "in", "&", "via", "be"])
    
    if splitted_term[0] in stop_words:
        stop_words = stop_words - set([splitted_term[0]])
                
    for sw in stop_words:
        while sw in splitted_term:
            splitted_term.remove(sw)
    sanitized_term = " ".join([w for w in splitted_term]) 
        
    return sanitized_term

In [246]:
def check_initial_letters(a, t):
    initial_letters_of_tokens_of_t = ''.join([c[0] for c in t.split()])
    if initial_letters_of_tokens_of_t == a or initial_letters_of_tokens_of_t.upper() == a:
        return True

In [247]:
def check_length_consistency(a, t):
    length_consistency = False
    if len(t.split()) <= len(a):
        length_consistency = True
    return length_consistency

In [248]:
def check_order(a, t):
    abbv_reversed = a.lower()[::-1]
    term_reversed = t.lower()[::-1]
    len_of_term = len(t)
    
    pos_memory = 0
    pos_memory_list = []
    order_matching_string_rev = ""
    
    for j, char_from_abbv in enumerate(abbv_reversed):
        if j == len(abbv_reversed) - 1:
            if char_from_abbv == term_reversed[-1]:
                order_matching_string_rev = order_matching_string_rev + char_from_abbv
                pos_memory_list.append(0)
        else:
            for i, char_from_term in enumerate(term_reversed[pos_memory:]):
                if char_from_abbv == char_from_term:
                    order_matching_string_rev = order_matching_string_rev + char_from_abbv
                    pos_memory = pos_memory + i + 1
                    pos_memory_list.append(len_of_term - pos_memory)
                    break
                
    if order_matching_string_rev == abbv_reversed:
        return True, pos_memory_list[::-1]
    else:
        return False, []

In [249]:
check_order("XMPP", "xtensibxle Messaging and Presence Protocol")

(True, [0, 11, 25, 34])

In [250]:
def check_distribution_of_matching_characters(pos_of_chars_list, t):
    term_intervals = []
    len_of_term = len(t)
    i = 0
    while i < len_of_term:
        sublist = []
        j = i
        while j < len_of_term and t[j] != " ":
            sublist.append(j)
            j = j+ 1
        i = j+1
        term_intervals.append(sublist)
        
    splitted_term = t.split()      
    
    containment_list = []
    for i, interval in enumerate(term_intervals):
        contanment_sublist = []
        for pos in pos_of_chars_list:
            if (pos in interval) and (splitted_term[i][0] == t[pos]):
                contanment_sublist.append(0)
            elif pos in interval:
                contanment_sublist.append(interval.index(pos))
        if len(contanment_sublist) == 0:
            contanment_sublist.append(-1)
        containment_list.append(contanment_sublist)
    
    result_of_distribution_check = False
    if len(containment_list) <= 1:
        result_of_distribution_check = True
    elif len (containment_list) >= 2:
        non_zero_count = 0
        for sublist in containment_list[1:]:
            if len(sublist) == 1 and 0 not in sublist:
                non_zero_count += 1
        if non_zero_count == 0:
            result_of_distribution_check = True
    
    return result_of_distribution_check

In [251]:
def base_algo(abbv, term):
    valid_order, pos_of_chars_list = check_order(abbv, term)
    return valid_order

In [252]:
def match_rating_comparison(a, term):
    if jellyfish.match_rating_comparison(a, term):
        return 1
    elif not jellyfish.match_rating_comparison(a, term):
        return 0

In [253]:
def check_wether_abbv_is_proper_short_form_of_term(abbv, term):
    if (abbv[0].lower() == term[0].lower()):
        
        
        ###################################### Step (a) ##########################################
        # check wether initial letters of tokens in t match with the letters in abbreviation
        if check_initial_letters(abbv, term):
            return True
        
        
        
        ###################################### Step (b) ########################################
        # clean abbreviation and term from special characters and stopwords
        sanitized_abbv, sanitized_term = clear_special_characters(abbv, term) 
        sanitized_term_without_stopswords = stop_words_handling(sanitized_term)
        sanitized_term_without_stopswords_splitted  = sanitized_term_wo_stopswords.split()
        
        # check wether initial letters of tokens in t' match with the letters in a'
        if check_initial_letters(sanitized_abbv, sanitized_term_without_stopswords):
            return True
        
        
        ###################################### Step (c) ##########################################
        # Sequential call of the methods that check and compare lengths, order and distribution of characters
        length_consistency = check_length_consistency(sanitized_abbv, sanitized_term_without_stopswords)
        order, pos_of_chars_list = check_order(sanitized_abbv, sanitized_term_without_stopswords)
        distribution = check_distribution_of_matching_characters(pos_of_chars_list, sanitized_term_without_stopswords)


        if length_consistency and order and distribution:
            return True
        else:
            return False

        ################################## in case nothing matches #################################
    else:
        return False

In [254]:
data = pd.read_csv('abbr_db.CSV', names=['abbr', 'long_forms'], sep=';', encoding='utf8')
abbr = list(data['abbr'].values)
long_forms = list(data['long_forms'].values)

In [255]:
def calc_fn(algo):
    FN = 0
    for i, abb in enumerate(abbr):
        if not algo(abb, long_forms[i]):
            print("\"" + abb + "\", " + "\"" + long_forms[i] + "\"")
            print("##################################")
            FN += 1
    return str(FN) + " FALSE NEGATIVES. Pairs that could not be detected out of " + str(len(abbr)) + " given pairs"

In [256]:
def calc_fp(algo):
    count_of_false_examples = 0
    FP = 0
    for i, abb in enumerate(abbr):
        len_ = len(abbr)
        for j in range (i, len_):
            if abb != abbr[j]:
                count_of_false_examples += 1
                if algo(abb, long_forms[j]):
                    print("\"" + abb + "\", " + "\"" + long_forms[j] + "\"")
                    FP +=1
    return str(FP) + " FALSE POSITIVE detections out of " +  str(count_of_false_examples) + " created false examples"

In [257]:
a = calc_fp(base_algo)
print(a)

"AA", "Advanced Audio Coding"
"AA", "autmoatic acoustic management"
"AA", "Auto Area Segmentation"
"AA", "as a service"
"AA", "Atanasoff-Berry Computer"
"AA", "abnormabl end"
"AA", "Advanced BIOS"
"AA", "AdBlock Plus"
"AA", "anonymous coward"
"AA", "Asheron's Call"
"AA", "alternating current"
"AA", "account"
"AA", "access control entry"
"AA", "acknowledgment"
"AA", "access control list"
"AA", "Association for Computing Machinery"
"AA", "Advanced Configuration and Power Interface"
"AA", "annual compliance report"
"AA", "actual cell rate"
"AA", "attenuation crosstalk ratio"
"AA", "absolute cell reference"
"AA", "access control system"
"AA", "Active Directory"
"AA", "Apple Desktop Bus"
"AA", "Android Debug Bridge"
"AA", "analog-to-digital"
"AA", "ActiveX Data Object"
"AA", "adaptive delta pulse code modulation"
"AA", "analog display service interface"
"AA", "Active Directory Service Interface"
"AA", "asymmetric digital subscriber line"
"AA", "ADSTAR Distributed Storage Management"
"AA", "

"ACPI", "annual compliance report"
"ACPI", "Advanced Encryption Standard"
"ACPI", "accelerated graphics port"
"ACPI", "aspect-oriented programming"
"ACPI", "application programming interface"
"ACPI", "Advanced Programmable Interrupt Controller"
"ACPI", "Automatic Private Internet Protocol Addressing"
"ACPI", "advanced persistent threat"
"ACPI", "advanced packaging tool"
"ACPI", "advanced processing unit"
"ACPI", "accelerated processing unit"
"ACPI", "authorized service provider"
"ACPI", "application service provider"
"ACPI", "advanced SCSI programming interface"
"ACPI", "automatic speech recognition"
"ACPI", "AT Attachment Packet Interface"
"ACPI", "acceptable use policy"
"ACR", "access control system"
"ACR", "Active Directory"
"ACR", "analog display service interface"
"ACR", "Active Directory Service Interface"
"ACR", "asymmetric digital subscriber line"
"ACR", "automatic exposure"
"ACR", "Advanced Encryption Standard"
"ACR", "accelerated graphics port"
"ACR", "Advanced Host Controlle

"AES", "American Standard Code for Information Interchange"
"AES", "American Society of Mechanical Engineers"
"AES", "autonomous sensory meridian response"
"AES", "answer set programming"
"AES", "authorized service provider"
"AES", "Active Server Pages"
"AES", "advanced SCSI programming interface"
"AES", "automated system recovery "
"AES", "advanced transfer cache"
"AES", "Adaptive Transform Acoustic Coding"
"AES", "acceptable use policy"
"AES", "automatic voice response"
"AES", "Amazon Web Services"
"AES", "Adobe Flash"
"AF", "Advanced Host Controller Interface"
"AF", "artificial intelligence"
"AF", "Artificial Intelligent Markup Language"
"AF", "Automatic Number Identification"
"AF", "application programming interface"
"AF", "Android package file"
"AF", "American Standard Code for Information Interchange"
"AF", "American Society of Mechanical Engineers"
"AF", "advanced SCSI programming interface"
"AF", "AT Attachment Packet Interface"
"AF", "advanced transfer cache"
"AF", "above the 

"AOSP", "as soon as possible"
"AOSP", "autonomous sensory meridian response"
"AOSP", "authorized service provider"
"AOSP", "application service provider"
"AOSP", "automatic speech recognition"
"AOSP", "automatic voice response"
"API", "Advanced Programmable Interrupt Controller"
"API", "Automatic Private Internet Protocol Addressing"
"API", "Android package file"
"API", "application"
"API", "advanced persistent threat"
"API", "advanced packaging tool"
"API", "advanced processing unit"
"API", "accelerated processing unit"
"API", "Application System/400"
"API", "as soon as possible"
"API", "answer set programming"
"API", "authorized service provider"
"API", "application service provider"
"API", "advanced SCSI programming interface"
"API", "automatic speech recognition"
"API", "AT Attachment Packet Interface"
"API", "Adaptive Transform Acoustic Coding"
"API", "acceptable use policy"
"APIC", "Automatic Private Internet Protocol Addressing"
"APIC", "application"
"APIC", "Application System/

"AVI", "automatic voice response"
"AVI", "automatic voice recognition"
"AVI", "Amazon Web Services"
"BAE", "Beginners All-purpose Symbolic Instruction Code"
"BAE", "bulletin board system"
"BAE", "binary-coded decimal"
"BAE", "best current practices"
"BAE", "Basic Combined Programming Language"
"BAE", "backup domain controller"
"BAE", "bidirectional override"
"BAE", "beyond economical repair"
"BAE", "bit error rate"
"BAE", "bit error rate test"
"BAE", "bit error rate tester"
"BAE", "Battlefield"
"BAE", "battleground"
"BAE", "bad game"
"BAE", "Border Gateway Multicast Protocol"
"BAE", "Border Gateway Protocol"
"BAE", "Berkeley Internet Name Domain"
"BAE", "basic input/output system"
"BAE", "because it's time network"
"BAE", "bold, italic, underline"
"BAE", "Bayonet Neill-Concelman"
"BAE", "back orifice"
"BAE", "Bastard Operator From Hell"
"BAE", "Berkeley Open Infrastructure for Network Computing"
"BAE", "beginning of message"
"BAE", "bill of materials"
"BAE", "BIOS pattern block"
"BAE",

...

"PSU", "pseudoterminal slave"
"PSU", "pseudoterminal"
"PSU", "physical unit"
"PSU", "PlayerUnknown's Battlegrounds"
"PSU", "PCI extensions for instrumentation"
"PTR", "pseudoterminal slave"
"PTR", "pseudoterminal"
"PTR", "PlayerUnknown's Battlegrounds"
"PTR", "print working directory"
"PTR", "preboot execution environment"
"PTR", "PCI extensions for instrumentation"
"PTS", "PlayerUnknown's Battlegrounds"
"PTS", "PCI extensions for instrumentation"
"PTY", "print working directory"
"PU", "PlayerUnknown's Battlegrounds"
"PU", "pickup group"
"PU", "preboot execution environment"
"PU", "PCI extensions for instrumentation"
"PVE", "Player vs. Player"
"PVE", "personal video recorder"
"PVE", "progressive web application"
"PVE", "Personal Web Server"
"PVE", "preboot execution environment"
"PVP", "progressive web application"
"PVR", "Personal Web Server"
"PVR", "preboot execution environment"
"PW", "progressive web application"
"PW", "password"
"PW", "print working directory"
"PW", "Personal Web 

"SaaS", "Standalone Server"
"SaaS", "Serial Attached SCSI"
"SaaS", "Security Administrator Tool for Analyzing Networks"
"SaaS", "Serial Communications Controller"
"SaaS", "Search for Extraterrestrial Intelligence"
"SaaS", "Self-Monitoring Analysis and Reporting Technology"
"SaaS", "system management bus"
"SaaS", "System Management Bus"
"SaaS", "system management server"
"SaaS", "storage management subsystem"
"SaaS", "Simple Mail Transfer Protocol"
"SaaS", "Scalar Processor Architecture"
"SaaS", "stateful packet inspection"
"SaaS", "spam over instant message"
"SaaS", "simultaneous peripheral operatings online"
"SaaS", "Standard Power Supply"
"SaaS", "static random access memory"
"SACD", "Serial Attached SCSI"
"SACD", "Secure Digital High Capacity card"
"SACD", "single edge contact cartridge"
"SACD", "Security-Enhanced Linux"
"SACD", "sealed lead-acid"
"SACD", "SmartMedia card"
"SACD", "Serial Presence Detection"
"SACD", "static random access memory"
"SACD", "storage service provider"
"S

"Sig", "symmetric multiprocessing"
"Sig", "Simple Network Management Protocol"
"Sig", "synchronous optical networking"
"Sig", "spelling"
"Sig", "Sony and Phillips Digital Interconnect Format"
"Sig", "spam over instant message"
"Sig", "systems programming language"
"Sig", "simultaneous peripheral operatings online"
"Sig", "Streaming SIMD Extensions"
"Sig", "small-scale integration"
"Sig", "Secure Socket Tunneling Protocol"
"Sig", "Spanning Tree Protocol"
"Sig", "synchronous transport signal"
"Sig", "Super Video Graphics Array"
"SIGINT", "single instruction, multiple data"
"SIGINT", "Self-Monitoring Analysis and Reporting Technology"
"SIGINT", "Sony and Phillips Digital Interconnect Format"
"SIGINT", "single instruction, multiple data"
"SIGINT", "Self-Monitoring Analysis and Reporting Technology"
"SIGINT", "Sony and Phillips Digital Interconnect Format"
"Sim", "Subscriber Identity Module card"
"Sim", "single instruction, multiple data"
"Sim", "single inline memory module"
"Sim", "service

"TA", "tape operating system"
"TA", "tracks per inch"
"TA", "Trusted Platform Module"
"TA", "Twisted-Pair Physical Medium Dependant"
"TA", "technical report"
"TA", "transfer resistance"
"TA", "Tripple Data Encryption Algorithm"
"TA", "Telecommunications Relay Service"
"TA", "terminate-and-stay-resident"
"TA", "transistor-transistor logic"
"TA", "trasmit data"
"Ta", "Terminal Access Controller Access Control System"
"Ta", "Telephony Application Programming Interface"
"Ta", "tape archive"
"Ta", "total cost of ownership"
"Ta", "Trusted Computer Platform Alliance"
"Ta", "Transmission Control Protocol/Internet Protocol"
"Ta", "table data"
"Ta", "telecommunications device for the deaf"
"Ta", "team deathmatch"
"Ta", "thermal design power"
"Ta", "telecommunications line"
"Ta", "The Exiled Ream of Arborea"
"Ta", "text enhancement technology"
"Ta", "Trivial File Transfer Protocol"
"Ta", "table head"
"Ta", "total harmonic distortion"
"Ta", "Titanium"
"Ta", "Textas Instruments"
"Ta", "Tagged Image

"W", "Western Digital"
"W", "Window Driver Foundation"
"W", "web application"
"W", "Wired Equivalent Privacy"
"W", "Wired for Management"
"W", "Windows File Protection"
"W", "Windows Genuine Advantage"
"W", "Worldwide Interoperability for Microwave Access"
"W", "Windows Pre-install Environment"
"W", "Windows socket"
"W", "wireless Internet service provider"
"W", "Windows metafile format"
"W", "Windows Management Instrumentation"
"W", "Website META Language"
"W", "Wireless Markup Language"
"W", "Wake-on-LAN"
"W", "word of the day"
"W", "World of Warcraft"
"W", "Windows on Windows"
"W", "WordPress"
"W", "word processor"
"W", "write pre-comp"
"W", "word processing competency"
"W", "Wide Quad Extended Graphics Array"
"W", "Windows RAM"
"W", "Widescreen Super eXtended Graphics Array"
"W", "what the hell"
"W", "World Wide Web"
"W", "what you see is what you get"
"W", "what you see is what you print"
"WAIS", "Web-based Enterprise Management"
"WAIS", "Wideband Code Division Multiple Access"
"W

In [258]:
b = calc_fn(base_algo)
print(b)

"ADC", "analog-to-digital"
##################################
"ASN.1", "abstract syntax number one"
##################################
"B2B", "business-to-business"
##################################
"B2B", "business-to-consumer"
##################################
"BDF", "Glyph Bitmap Distribution Format"
##################################
"Bi-di", "bidirectional"
##################################
"CDA", "compact audio disc"
##################################
"CD-i", "Compact Disc Interactive"
##################################
"CD-R", "Compact Disc Recordable"
##################################
"CD-ROM", "Compact Disc Read-Only Memory"
##################################
"CD-RW", "Compact Disc Re-Writable"
##################################
"CD-WO", "Compact Disc Write Once"
##################################
"CLCC", "ceramic leadless lead carrier"
##################################
"Code", "source code"
##################################
"Coil", "electromagnetic coil"
###############

In [259]:
a = calc_fp(check_wether_abbv_is_proper_short_form_of_term)
print(a)

"AA", "account"
"AA", "acknowledgment"
"AA", "analog-to-digital"
"AA", "Autofocus"
"AA", "All-in-One"
"AA", "alphabet"
"AA", "ampere"
"AA", "application"
"AA", "argument"
"AA", "automobile"
"AA", "autonomous"
"AA", "automatic"
"AA", "avatar"
"AA", "antivirus"
"AAC", "Atanasoff-Berry Computer"
"AAC", "anonymous coward"
"AAC", "Asheron's Call"
"AAC", "alternating current"
"AAC", "account"
"AAC", "acknowledgment"
"AAC", "Autofocus"
"AAC", "application"
"AAC", "automatic"
"AAM", "acknowledgment"
"AAM", "ante meridiem"
"AAM", "amplitude modulation"
"AAM", "ampere"
"AAM", "aspect-oriented programming"
"AAM", "argument"
"AAM", "automobile"
"AAM", "autonomous"
"AAM", "automatic"
"AAS", "Autofocus"
"AAS", "autonomous system"
"AAS", "Application System/400"
"AAS", "autonomous"
"AAS", "antivirus"
"AAS", "Adobe Flash"
"AAS", "Autofocus"
"AAS", "autonomous system"
"AAS", "Application System/400"
"AAS", "autonomous"
"AAS", "antivirus"
"AAS", "Adobe Flash"
"AC", "account"
"AC", "acknowledgment"
"AC",

"COBOL", "Component Object Model"
"Code", "Compression / Decompression"
"Code", "COmpression / DECompression"
"Code", "Computer Dealers' Exhibition"
"Code", "confidence"
"Codec", "confidence"
"Codec", "confidence"
"Coil", "Combined Programming Language"
"Coil", "Certificate Revocation List"
"Con", "counter-strike"
"Con", "counter-strike"
"Con", "counter-strike"
"Core", "customer service"
"Core", "counter-strike"
"Core", "click-through rate"
"CPA", "cryptography"
"CPA", "cryptography"
"CPAN", "Combined Programming Language"
"CPAN", "cost per thousand"
"CPC", "client-to-client protocol"
"CPGA", "Combined Programming Language"
"CPGA", "cryptography"
"CPI", "cross-site scripting"
"CPL", "client-to-client protocol"
"CPS", "computer science"
"CR", "continuity RIMM"
"CR", "cryptography"
"CR", "counter-strike"
"CR", "click-through rate"
"CRC", "customer service"
"CRC", "client-to-client protocol"
"CREN", "customer relationship management"
"CREN", "computer science"
"C-RIMM", "customer relation

"EFS", "end-of-message"
"EFS", "End-of-Transmission"
"EGA", "Extended Graphics Array"
"EIA", "Extended ISA"
"EIA", "electroluminescent display"
"EIA", "electronic publication"
"EIA", "electrostatic discharge"
"EIDE", "electrostatic discharge"
"EISA", "electroluminescent display"
"EISA", "electrostatic discharge"
"EISA", "End-User License Agreement"
"EISA", "electroluminescent display"
"EISA", "electrostatic discharge"
"EISA", "End-User License Agreement"
"ELD", "electrostatic discharge"
"ELF", "electromotive force"
"EM", "electronic mail"
"EM", "enhanced metafile"
"EM", "expanded-memory manager"
"EM", "end-of-message"
"EM", "End-of-Transmission"
"E-mail", "enhanced metafile"
"EMI", "End-of-Transmission"
"EMI", "Evolution-Data Optimized"
"EMS", "end-of-message"
"EMS", "End-of-Transmission"
"EMS", "extended memory specification"
"EMS", "end-of-message"
"EMS", "End-of-Transmission"
"EMS", "extended memory specification"
"EOF", "End-of-Line"
"EOF", "end-of-message"
"EOF", "end-of-page"
"EO

"mb", "mebibit"
"mb", "mebibyte"
"mb", "mobile blog"
"Mb", "megabyte"
"Mb", "Malwarebytes"
"Mb", "Managed Beans"
"Mb", "mebibit"
"Mb", "mebibyte"
"Mb", "mobile blog"
"MB", "Malwarebytes"
"MB", "Managed Beans"
"MB", "mebibit"
"MB", "mebibyte"
"MB", "mobile blog"
"MBAM", "Multibank DRAM"
"MBean", "mobile identification number"
"MBL", "mobile blog"
"Mbps", "megabytes per second"
"MBR", "model number"
"MC", "MultiMediaCard"
"MC", "metal-oxide-semiconductor"
"MC", "Microsoft"
"MC", "millisecond"
"MC", "microarchitecture"
"MCA", "multi-factor authentication"
"MCA", "MultiMediaCard"
"MCA", "microarchitecture"
"MCSE", "magneto-optical diskette"
"MCSE", "Microsoft Cluster Server"
"MDA", "Multibank DRAM"
"MDA", "man-in-the-middle attack"
"MDA", "MultiMediaCard"
"MDA", "modulator/demodulator"
"MDA", "Mozilla Foundation"
"MDA", "Microsoft Diagnostics"
"MDC", "make directory"
"MDC", "MultiMediaCard"
"MDC", "metal-oxide-semiconductor"
"MDC", "Microsoft Diagnostics"
"MDI", "make directory"
"MDI", "Mu

"PP", "Pagerank"
"PP", "printer"
"PP", "PowerShell"
"PP", "PlayStation Portable"
"PP", "pseudoterminal"
"PP", "password"
"PP", "password"
"PP", "pixel"
"PPT", "printer"
"PPT", "PlayStation Portable"
"PPT", "pseudoterminal"
"PPTP", "PlayStation Portable"
"PR", "Parameter RAM"
"PR", "printer"
"PR", "PowerShell"
"PR", "pseudoterminal"
"PR", "password"
"PR", "password"
"PR", "Parameter RAM"
"PR", "printer"
"PR", "PowerShell"
"PR", "pseudoterminal"
"PR", "password"
"PR", "password"
"PRAM", "Private Message"
"PRAM", "programmable read-only memory"
"PRN", "print screen"
"PRN", "pseudoterminal"
"PRN", "PlayerUnknown's Battlegrounds"
"Proglog", "Programming in Logic"
"PS", "Personal System/2"
"PS", "Personal Store"
"PS", "pseudoterminal slave"
"PS", "pseudoterminal"
"PS", "password"
"PS", "password"
"PS", "Personal System/2"
"PS", "Personal Store"
"PS", "pseudoterminal slave"
"PS", "pseudoterminal"
"PS", "password"
"PS", "password"
"PS", "Personal System/2"
"PS", "Personal Store"
"PS", "pseudot

"SOA", "software"
"SOA", "system operator"
"SOA", "Solitaire"
"SOA", "super operator"
"SOA", "Suspend-to-RAM"
"SOA", "software"
"SOA", "system operator"
"SoC", "source code"
"SOI", "Solitaire"
"SOI", "synchronize"
"SOI", "synchronize"
"SONET", "Spam over Internet Telephony"
"SOP", "system operator"
"SP", "specification"
"SP", "Suspend-to-RAM"
"SP", "subscript"
"SP", "superscript"
"SPARC", "system packet interface"
"SPARC", "serial peripheral interface"
"SPD", "Suspend-to-RAM"
"Spec", "system packet interface"
"Spec", "serial peripheral interface"
"Spec", "supertwisted nematic"
"Spec", "superscript"
"SPI", "superscript"
"SPI", "superscript"
"SPI", "superscript"
"Spit", "superscript"
"Spool", "Synchronous Transport Module"
"SPP", "superscript"
"SPS", "superscript"
"SRAM", "Synchronous Transport Module"
"SRAM", "Suspend-to-RAM"
"SS", "Suspend-to-RAM"
"SS", "subscript"
"SS", "superscript"
"SS", "software"
"SS", "synchronize"
"SS", "synchronize"
"SS", "synthesizer"
"SSD", "Suspend-to-RAM"
"

In [260]:
b = calc_fn(check_wether_abbv_is_proper_short_form_of_term)
print(b)

"ADC", "analog-to-digital"
##################################
"AIX", "advanced interactive executive"
##################################
"APK", "Android package file"
##################################
"ARCnet", "Attached Resource Computer network"
##################################
"ARPANET", "Advanced Research Projects Agency Network"
##################################
"ARQ", "automatic repeat request"
##################################
"ASN.1", "abstract syntax number one"
##################################
"ATX", "Advanced Technology eXtended"
##################################
"B2B", "business-to-business"
##################################
"B2B", "business-to-consumer"
##################################
"Basic", "Beginners All-purpose Symbolic Instruction Code"
##################################
"BDF", "Glyph Bitmap Distribution Format"
##################################
"Br", "Break key"
##################################
"BTX", "Balanced Technology eXtended"
###################

In [261]:
a = calc_fp(match_rating_comparison)
print(a)

"AAC", "account"
"AAC", "icon"
"AAC", "receive"
"AAM", "ampere"
"AAM", "maximum"
"AAM", "minimum"
"AAM", "Titanium"
"AAS", "Celsius"
"AAS", "release"
"AAS", "user"
"AAS", "versus"
"AAS", "Celsius"
"AAS", "release"
"AAS", "user"
"AAS", "versus"
"ABC", "automobile"
"ABC", "automatic"
"ABC", "kibibit"
"ABC", "mebibit"
"ABC", "Tebi"
"Abend", "Advanced BIOS"
"Abend", "account"
"Abend", "absolute cell reference"
"Abend", "ampere"
"Abend", "as soon as possible"
"Abend", "autonomous system number"
"Abend", "above the fold"
"Abend", "Advanced Technology eXtended"
"Abend", "autonomous"
"Abend", "avatar"
"Abend", "broadband"
"Abend", "class ID"
"Abend", "source code"
"Abend", "download"
"Abend", "download"
"Abend", "Gigabits per second"
"Abend", "kibibyte"
"Abend", "line feed"
"Abend", "memorandum"
"Abend", "mebibyte"
"Abend", "NOT NULL"
"Abend", "nanosecond"
"Abend", "Record"
"Abend", "source code"
"Abend", "table data"
"Abend", "table head"
"Abend", "upload"
"Abend", "upload"
"Abend", "XP mode"

"ARPANET", "service-level agreement"
"ARPANET", "super-large scale integration"
"ARPANET", "Serial Presence Detection"
"ARPANET", "Source Service Access Point"
"ARPANET", "standard input"
"ARPANET", "terminal adapter"
"ARPANET", "terminate-and-stay-resident"
"ARPANET", "user agent"
"ARPANET", "user-defined function"
"ARPANET", "user-defined font"
"ARPANET", "User State Migration Tool"
"ARPANET", "unique selling point"
"ARPANET", "Microsoft Visual Basic Scripting Edition"
"ARPANET", "Version"
"ARPANET", "very-large-scale integration"
"ARPANET", "Virtual Machine Monitor"
"ARPANET", "wireless access point"
"ARPANET", "Warning, Advice and Reporting Point"
"ARPANET", "web application"
"ARPANET", "Wired for Management"
"ARQ", "Core 2"
"ARQ", "versus"
"AS", "Celsius"
"AS", "release"
"AS", "user"
"AS", "versus"
"ASAP", "Google Plus"
"ASAP", "page up"
"ASAP", "user"
"ASCII", "automatic"
"ASCII", "nanosecond"
"ASCII", "user"
"ASME", "automobile"
"ASME", "automatic"
"ASME", "loss of frame"
"ASME"

"BITNET", "Standard Parallel Port"
"BITNET", "standard input"
"BITNET", "toolkit"
"BITNET", "Turing test"
"BNC", "Boolean"
"BNC", "bitcoin"
"BNC", "function"
"BNC", "manual"
"BNC", "minimum"
"BNC", "Titanium"
"BOFH", "Boolean"
"BOINC", "Boolean"
"BOINC", "bitcoin"
"BOINC", "function"
"BOINC", "manual"
"BOINC", "minimum"
"BOINC", "Titanium"
"BOM", "Boolean"
"BOM", "maximum"
"BOM", "minimum"
"BOM", "Titanium"
"BOM", "Boolean"
"BOM", "maximum"
"BOM", "minimum"
"BOM", "Titanium"
"BOM", "Boolean"
"BOM", "maximum"
"BOM", "minimum"
"BOM", "Titanium"
"BOM", "Boolean"
"BOM", "maximum"
"BOM", "minimum"
"BOM", "Titanium"
"Bool", "Celsius"
"Bool", "level"
"Bool", "milli"
"Bool", "manual"
"Bool", "modulo"
"Bool", "pixel"
"Bool", "release"
"BPB", "opinion"
"BPB", "Tebi"
"BPDU", "core dump"
"BPDU", "opinion"
"BPDU", "Super Video"
"BPDU", "upload"
"BPDU", "upload"
"BPDU", "XP mode"
"BPL", "level"
"BPL", "milli"
"BPL", "manual"
"BPL", "modulo"
"BPL", "opinion"
"BPL", "pixel"
"BPL", "spelling"
"BPL", "t

"CentOS", "intrusion detection system"
"CentOS", "Internet Engineering Notes"
"CentOS", "International Federation for Information Processing"
"CentOS", "Internet Movie Database"
"CentOS", "International Committee for Information Technology Standards"
"CentOS", "integer"
"CentOS", "Internetworking operating system"
"CentOS", "intrusion prevention system"
"CentOS", "Internet Protocol version 4"
"CentOS", "Internet Protocol version 6"
"CentOS", "integrated software for imagers and spectrometers"
"CentOS", "one thousand"
"CentOS", "line feed"
"CentOS", "media access unit"
"CentOS", "maximum transmission unit"
"CentOS", "nanometer"
"CentOS", "picture"
"CentOS", "Pentium 2"
"CentOS", "Pantone Matching System"
"CentOS", "packet over SONET"
"CentOS", "printer"
"CentOS", "Request for Comments"
"CentOS", "Reseaux IP Europeens notes"
"CentOS", "single-density diskette"
"CentOS", "single-sided diskette"
"CentOS", "sound effects"
"CentOS", "signature"
"CentOS", "Solitaire"
"CentOS", "Tantalum"
"Cen

"CSMA/CD", "Ultimate Boot CD"
"CSMA/CD", "User Datagram Protocol"
"CSMA/CD", "universal product code"
"CSMA/CD", "Unix-to-Unix encode"
"CSMA/CD", "valve-regulated lead-acid"
"CSMA/CD", "Visual Studio"
"CSNET", "Channel Service Unit/Data Service Unit"
"CSNET", "client-to-client protocol"
"CSNET", "Clear to Send"
"CSNET", "change teletype"
"CSNET", "control unit"
"CSNET", "Drive Active Slave Present"
"CSNET", "distributed computing environment"
"CSNET", "Destination Service Access Point"
"CSNET", "drive secondary present"
"CSNET", "evaluate"
"CSNET", "favorite"
"CSNET", "function"
"CSNET", "height"
"CSNET", "kilobit"
"CSNET", "kilobit"
"CSNET", "kibibyte"
"CSNET", "logic unit"
"CSNET", "media access unit"
"CSNET", "megabit"
"CSNET", "mebibyte"
"CSNET", "maximum transmission unit"
"CSNET", "National Center for Supercomputing Applications"
"CSNET", "NOT NULL"
"CSNET", "no operatation"
"CSNET", "nanosecond"
"CSNET", "operation"
"CSNET", "open shortest path first"
"CSNET", "petabit"
"CSNET",

...

"INCITS", "Windows File Protection"
"Inetd", "information"
"Inetd", "index node"
"Inetd", "integer"
"Inetd", "interrupt"
"Inetd", "Internet Obfuscated C Code Contest"
"Inetd", "input/output operations per second"
"Inetd", "Internetworking operating system"
"Inetd", "Internet Protocol"
"Inetd", "ingress protection"
"Inetd", "intellectual property"
"Inetd", "interprocess communication"
"Inetd", "interpupillary distance"
"Inetd", "initial program load"
"Inetd", "Internet Protocol next generation"
"Inetd", "intrusion prevention system"
"Inetd", "Internet Protocol security"
"Inetd", "Internet Protocol television"
"Inetd", "Internet Protocol version 4"
"Inetd", "Internet Protocol version 6"
"Inetd", "Internetwork Packet Exchange/Sequenced Packet Exchange"
"Inetd", "infrared"
"Inetd", "Internet Relay Chat"
"Inetd", "Infrared Data Association"
"Inetd", "interrecord gap"
"Inetd", "Intensity-Red-Green-Blue"
"Inetd", "interrupt request"
"Inetd", "Industry Standard Architecture"
"Inetd", "Informat

"LFAP", "level"
"LFAP", "page up"
"LFD", "level"
"LFG", "level"
"LFM", "loss of frame"
"LFM", "level"
"LFM", "maximum"
"LFM", "minimum"
"LFM", "Titanium"
"LFN", "level"
"LFN", "opinion"
"LG", "level"
"LG", "page up"
"LGA", "level"
"LGA", "page up"
"LGPL", "Logical Link Control"
"LGPL", "Loss of signal"
"LGPL", "line printer terminal"
"LGPL", "logic unit"
"LGPL", "level"
"LGPL", "megabit"
"LGPL", "megaflop"
"LGPL", "not equal"
"LGPL", "not gonna lie"
"LGPL", "NOT NULL"
"LGPL", "page up"
"LGPL", "top of file"
"LGPL", "voicemail"
"LH", "level"
"LIFO", "level"
"Li-ion", "level"
"Li-ion", "opinion"
"Li-polymer", "list server"
"Li-polymer", "loss of frame"
"Li-polymer", "line printer daemon"
"Li-polymer", "line printer terminal"
"Li-polymer", "Low Profile eXtension"
"Li-polymer", "least significant character"
"Li-polymer", "Layered Service Provider"
"Li-polymer", "non-impact printer"
"Li-polymer", "non-printing character"
"Li-polymer", "non-playable character"
"Li-polymer", "non-return-to-ze

"MTBF", "not equal"
"MTBF", "NOT NULL"
"MTBF", "Not-Or"
"MTBF", "number"
"MTBF", "out of office"
"MTBF", "petabit"
"MTBF", "petabyte"
"MTBF", "retweet"
"MVNO", "opinion"
"MVP", "page up"
"MVS", "release"
"MVS", "versus"
"MX record", "network interface card"
"MX record", "nickel-metal hydride"
"MX record", "OverDrive Processor"
"MX record", "overpowered"
"MX record", "on-screen keyboard"
"MX record", "one-time password"
"MX record", "overtype mode"
"MX record", "printed circuit board"
"MX record", "Microsoft Plus!"
"MX record", "print screen"
"MX record", "Resource Description Framework"
"MX record", "Recording Industry Association of America"
"MX record", "Super Audio CD"
"MX record", "storage area network"
"MX record", "Scroll Lock key"
"MX record", "Secure Digital High Capacity card"
"MX record", "safe for work"
"MX record", "Subscriber Identity Module card"
"MX record", "SmartMedia card"
"MX record", "Super Video CD"
"MX record", "Super Video Graphics Array"
"MX record", "Super Exte

"P-machine", "Small Scale Experimental Machine"
"P-machine", "tape archive"
"P-machine", "user-defined function"
"P-machine", "Unique Device Identification"
"P-machine", "universal Turing machine"
"P-machine", "Valve Anti-Cheat"
"P-machine", "Video Electronics Standard Association"
"P-machine", "virtual machine"
"P-machine", "web application"
"P-machine", "Windows File Protection"
"P-machine", "extended memory specification"
"PMF", "pixel"
"PMS", "pixel"
"PMS", "release"
"PMS", "versus"
"PN", "pixel"
"PN", "Titanium"
"PNG", "Pagerank"
"PNG", "pixel"
"PNG", "spelling"
"PNG", "Titanium"
"PnP", "Pagerank"
"PnP", "pixel"
"PnP", "Titanium"
"POD", "pixel"
"PoS", "pixel"
"PoS", "release"
"PoS", "user"
"PoS", "versus"
"POS", "pixel"
"POS", "release"
"POS", "user"
"POS", "versus"
"POSIX", "password"
"POSIX", "password"
"POSIX", "pixel"
"POSIX", "user"
"POST", "password"
"POST", "password"
"POST", "pixel"
"POST", "user"
"POTS", "printer"
"POTS", "pixel"
"POTS", "release"
"POTS", "set user ID"
"P

"SERP", "source code"
"SERP", "set user ID"
"SERP", "versus"
"Servlet", "single instruction, multiple data"
"Servlet", "super input/output"
"Servlet", "service-level agreement"
"Servlet", "secure login"
"Servlet", "super-large scale integration"
"Servlet", "source memory address register"
"Servlet", "server message block"
"Servlet", "storage management subsystem"
"Servlet", "Surface-mount Technology"
"Servlet", "service-oriented architecture"
"Servlet", "start of authority"
"Servlet", "silicon on insulator"
"Servlet", "super operator"
"Servlet", "source code"
"Servlet", "Serial Presence Detection"
"Servlet", "Structured Query Language"
"Servlet", "Source Service Access Point"
"Servlet", "service set identifier"
"Servlet", "Secure Sockets Layer"
"Servlet", "storage service provider"
"Servlet", "supertwisted nematic"
"Servlet", "superscript"
"Servlet", "Super XGA Plus"
"Servlet", "Universal Disk Format"
"Servlet", "unordered list"
"Servlet", "User State Migration Tool"
"Servlet", "Univer

In [262]:
b = calc_fn(match_rating_comparison)
print(b)

"AA", "Anti-alias"
##################################
"AAC", "Advanced Audio Coding"
##################################
"AAM", "autmoatic acoustic management"
##################################
"AAS", "Auto Area Segmentation"
##################################
"AAS", "as a service"
##################################
"ABC", "Atanasoff-Berry Computer"
##################################
"ABIOS", "Advanced BIOS"
##################################
"ABP", "AdBlock Plus"
##################################
"AC", "anonymous coward"
##################################
"AC", "Asheron's Call"
##################################
"AC", "alternating current"
##################################
"ACE", "access control entry"
##################################
"Ack", "acknowledgment"
##################################
"ACL", "access control list"
##################################
"ACM", "Association for Computing Machinery"
##################################
"ACPI", "Advanced Configuration and Power Inter

##################################
"mAH", "milliamp hour"
##################################
"MAPI", "Message Application Programming Interface"
##################################
"MAPS", "Mail Abuse Prevention Systems"
##################################
"MAR", "memory address register"
##################################
"Matlab", "matrix laboratory"
##################################
"MAU", "media access unit"
##################################
"mb", "motherboard"
##################################
"Mb", "megabit"
##################################
"MB", "megabyte"
##################################
"MBAM", "Malwarebytes"
##################################
"MBean", "Managed Beans"
##################################
"MBL", "microcomputer-based labs"
##################################
"Mbps", "megabits per second"
##################################
"MBps", "megabytes per second"
##################################
"MBR", "master boot record"
##################################
"MC", "mega

In [ ]:
######################### Experiments with syntactic and semantic similarity  ###################################################

In [229]:
def dice_coefficient(a, b):
    """dice coefficient 2nt/(na + nb)."""
    a_bigrams = set(a.lower())
    b_bigrams = set(b.lower())
    overlap = len(a_bigrams & b_bigrams)
    return overlap * 2.0 / (len(a_bigrams) + len(b_bigrams))

In [233]:
from random import randint
random_pairs = []
for i in range (0, 10):
    rd = randint(0, len(abbr))
    abbv = abbr[rd]
    term = long_forms[rd]
    random_pairs.append([abbv, term])

for pair in random_pairs:
    print("++++++++++++++++++++++++++++++++++++++++++")
    print("\"" + pair[0] + "\", " + "\"" + pair[1] + "\"")
    for j, measure in enumerate([jellyfish.levenshtein_distance, jellyfish.damerau_levenshtein_distance, jellyfish.hamming_distance, jellyfish.jaro_similarity, jellyfish.jaro_winkler_similarity, jellyfish.match_rating_comparison, dice_coefficient]):
        print(measure(pair[0], pair[1]))
    

++++++++++++++++++++++++++++++++++++++++++
"LED monitor", "light-emitting diode"
17
17
20
0.4348484848484849
0.4348484848484849
True
0.8181818181818182
++++++++++++++++++++++++++++++++++++++++++
"Int", "integer"
5
5
5
0.6507936507936508
0.6507936507936508
True
0.6666666666666666
++++++++++++++++++++++++++++++++++++++++++
"PS/2", "Personal System/2"
13
13
16
0.4362745098039216
0.4362745098039216
True
0.4444444444444444
++++++++++++++++++++++++++++++++++++++++++
"IANA", "Internet Assigned Numbers Authority"
31
31
34
0.611904761904762
0.611904761904762
None
0.3157894736842105
++++++++++++++++++++++++++++++++++++++++++
"SMM", "System Management Mode"
19
19
21
0.5858585858585859
0.5858585858585859
None
0.3076923076923077
++++++++++++++++++++++++++++++++++++++++++
"U/L", "upload"
6
6
6
0.0
0.0
True
0.4444444444444444
++++++++++++++++++++++++++++++++++++++++++
"IAP", "Internet access provider"
23
23
23
0.4583333333333333
0.4583333333333333
None
0.375
++++++++++++++++++++++++++++++++++++++++++

In [ ]:
########################### comparison on abbreviation level #######################################################

In [238]:
for k, abb in enumerate(abbr):
    abb_lower = abb.lower()
    term_lower = long_forms[k].lower()
    sanitized_abbv, sanitized_term = clear_special_characters(abb_lower, term_lower) 
    sanitized_term_without_stopswords = stop_words_handling(sanitized_term)
    initial_letters_of_tokens_of_sanitized_term_without_stopswords = ''.join([c[0] for c in sanitized_term_without_stopswords.split()])
    tmp_sum = 0
    for measure in [jellyfish.levenshtein_distance, jellyfish.damerau_levenshtein_distance, jellyfish.hamming_distance, jellyfish.jaro_similarity, jellyfish.jaro_winkler_similarity, match_rating_comparison, dice_coefficient]:
        tmp_sum = tmp_sum + measure(initial_letters_of_tokens_of_sanitized_term_without_stopswords, sanitized_abbv)
    print(tmp_sum/len(abbr))
        

0.0036192544335866814
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.0038122813367113036
0.006879168605552971
0.00660772452303397
0.0038122813367113036
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.005971769815418024
0.002171552660152009
0.00437326577391724
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.00437326577391724
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.008805129000568741
0.002171552660152009
0.0034382917119073473
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.004916153938955242
0.0033779708046809023
0.004252623

0.004916153938955242
0.004916153938955242
0.0053384002895403545
0.002171552660152009
0.0038122813367113036
0.0038122813367113036
0.002171552660152009
0.003632180342278062
0.01651931130758492
0.00437326577391724
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.012353628998580139
0.002171552660152009
0.005066956207021353
0.0034382917119073473
0.0038122813367113036
0.016537208719619143
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.0038122813367113036
0.009009681505609844
0.016442930938044942
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.004717956672354067
0.002171552660152009
0.002171552660152009
0.006659428157799494
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.002171552660152009
0.0034382917119073473
0.0034382917119073473
0.0038122813367113036
0.002171552660152009
0.002171552660152009
0.0021

In [242]:
 for measure in [jellyfish.levenshtein_distance, jellyfish.damerau_levenshtein_distance, jellyfish.hamming_distance, jellyfish.jaro_similarity, jellyfish.jaro_winkler_similarity, match_rating_comparison, dice_coefficient, check_wether_abbv_is_proper_short_form_of_term]:
    tmp_sum = 0
    for k, abb in enumerate(abbr):
        abb_lower = abb.lower()
        term_lower = long_forms[k].lower()
        sanitized_abbv, sanitized_term = clear_special_characters(abb_lower, term_lower) 
        sanitized_term_without_stopswords = stop_words_handling(sanitized_term)
        initial_letters_of_tokens_of_sanitized_term_without_stopswords = ''.join([c[0] for c in sanitized_term_without_stopswords.split()])
        tmp_sum = tmp_sum + measure(initial_letters_of_tokens_of_sanitized_term_without_stopswords, sanitized_abbv)
    print(tmp_sum/len(abbr))

0.8517915309446255
0.8501628664495114
1.1346362649294246
0.895096960284256
0.8961961651972516
0.9142236699239956
0.8650353883660081
0.9272529858849077
